## Personalized Learning (Localized Learning?)

#### This notebook includes the following online models;
1. A single global model with all data
2. Multiple local models (starting from a single global model)
   1. that are updated with new data
   2. that exchanges data in clusters
   3. that exchanges parameters in clusters

  
#### The dataset that is used for this project is [CIFAR-100 dataset][1]
* Has 100 classes containing 600 images each

#### New data are fed by the following rules;
1. Distributed, according to superclasses
  * Clusters will only be updated with data that belongs to a specific superclass
  * We update the NN by
    1. Changing all parameters of the NN
    2. Only changing the last few layers, as in many MTL models
2. Randomly (why?)

#### We expect to find an answer to the following research questions with this project;
1. If models are updated with data (or parameters) that are shared within a cluster, can the model perform good enough with the labels that count?
  * For example, the performance of the cluster that are updated with "Vehicles" superclass is only assessed with the labels that corresponds to the superclass.
  
[1]: https://www.cs.toronto.edu/~kriz/cifar.html

#### Questions

Retraining: how does it work <br>
How do we compare these models?


### Implementation with [MobileNet][2]

[2]: https://arxiv.org/abs/1704.04861

In [124]:
import pandas as pd
import numpy as np
import os
import keras
import datetime
import pickle
import matplotlib.pyplot as plt
from keras.datasets import cifar100
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from math import ceil

In [2]:
from keras.utils import to_categorical

In [3]:
# Hyperparameters
batch_size = 32
num_classes = 100
epochs = 30
data_augmentation = True
num_predictions = 20

#### Load CIFAR-100 dataset

In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [6]:
y_train_cat = keras.utils.to_categorical(
    y_train,
    num_classes=None,
    dtype='float32'
)
y_test_cat = keras.utils.to_categorical(
    y_test,
    num_classes=None,
    dtype='float32'
)

#### We use 35000 samples to train global model and 15000 samples to simulate data being updated locally

In [10]:
global_sample_num = 35000

In [99]:
# it seems like the training data is already in random order
x_train_global = x_train[:global_sample_num]
x_train_local = x_train[global_sample_num:]
y_train_global = y_train_cat[:global_sample_num]
y_train_local = y_train_cat[global_sample_num:]

In [109]:
y_train_local_notcat = y_train[global_sample_num:]

In [9]:
def fit_model_global(model, epochs):
    now = datetime.datetime.now()
    print ("Training date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))
    return model.fit(x_train_global, y_train_global,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(x_test, y_test_cat),
                      shuffle=True)

#### A few helper functions

In [91]:
def compile_model(model):  
    # initiate SGD optimizer
    opt = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

In [11]:
def compile_model_rms(model):  
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [97]:
def print_datetime():
    now = datetime.datetime.now()
    print ("Training date and time : ")
    print (now.strftime("%Y-%m-%d %H:%M:%S"))

#### Build custom MobileNet model with additional dense layers

In [12]:
base_model = MobileNet(weights=None, input_shape=x_train.shape[1:], include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
# @TODO maybe try initializing weights by weights='imagenet'?
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) # we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024,activation='relu')(x) # dense layer 2
x = Dense(512,activation='relu')(x) # dense layer 3
preds = Dense(y_train_cat.shape[1],activation='softmax')(x) # final layer with softmax activation

W0801 01:19:32.668441 139671577802496 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0801 01:19:32.702611 139671577802496 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0801 01:19:32.716075 139671577802496 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0801 01:19:32.746716 139671577802496 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The n

In [13]:
model=Model(inputs=base_model.input,outputs=preds)


In [14]:
# sgd = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])

# Let's train the model using RMSprop
compile_model(model)


W0801 01:19:39.753719 139671577802496 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



#### Global (1) Train globally on all 50k

idea) set batch_size differently?

In [24]:
now = datetime.datetime.now()
print ("Training date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))
history_G1 = model.fit(x_train, y_train_cat,
              batch_size=32,
              epochs=30,
              validation_data=(x_test, y_test_cat),
              shuffle=True)

Training date and time : 
2019-07-28 21:29:18
Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 49s 981us/step - loss: 0.0099 - acc: 0.0126 - val_loss: 0.0099 - val_acc: 0.0140
Epoch 2/30
50000/50000 [==============================] - 44s 883us/step - loss: 0.0099 - acc: 0.0156 - val_loss: 0.0099 - val_acc: 0.0193
Epoch 3/30
50000/50000 [==============================] - 44s 876us/step - loss: 0.0099 - acc: 0.0188 - val_loss: 0.0099 - val_acc: 0.0165
Epoch 4/30
50000/50000 [==============================] - 44s 875us/step - loss: 0.0099 - acc: 0.0192 - val_loss: 0.0099 - val_acc: 0.0182
Epoch 5/30
50000/50000 [==============================] - 44s 873us/step - loss: 0.0099 - acc: 0.0206 - val_loss: 0.0099 - val_acc: 0.0206
Epoch 6/30
50000/50000 [==============================] - 43s 869us/step - loss: 0.0099 - acc: 0.0208 - val_loss: 0.0099 - val_acc: 0.0229
Epoch 7/30
50000/50000 [==============================] - 44s 870us/st

Save model

In [25]:
model.save("model_G1_sgd.h5")

#### Sanity Check

In [18]:
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D}):
    model = keras.models.load_model("model_G1_epoch_30_keras_downgraded.h5")

In [19]:
compile_model(model)

In [20]:
model.evaluate(x_test, y_test_cat)

10000/10000 [==============================] - 4s 414us/step


[3.372575284194946, 0.2468]

<span style="color:red">Why does the accuracy drop?</span>
<br> it's not dropping. we should check the validation accuracy

#### Global (2): Train Globally on 15K + fix layers, continue on remaining 35K

In [26]:
batch_size_G2 = batch_size

In [27]:
base_model = MobileNet(weights=None, input_shape=x_train.shape[1:], include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
# @TODO maybe try initializing weights by weights='imagenet'?
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) # we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024,activation='relu')(x) # dense layer 2
x = Dense(512,activation='relu')(x) # dense layer 3
preds = Dense(y_train_cat.shape[1],activation='softmax')(x) # final layer with softmax activation

In [28]:
model_G2=Model(inputs=base_model.input,outputs=preds)

compile_model(model_G2)

now = datetime.datetime.now()
print ("Training date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

model_G2.fit(x_train_global, y_train_global,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test_cat),
              shuffle=True)

Training date and time : 
2019-07-28 21:57:47
Train on 35000 samples, validate on 10000 samples
Epoch 1/30
35000/35000 [==============================] - 37s 1ms/step - loss: 0.0099 - acc: 0.0117 - val_loss: 0.0099 - val_acc: 0.0158
Epoch 2/30
35000/35000 [==============================] - 31s 892us/step - loss: 0.0099 - acc: 0.0151 - val_loss: 0.0099 - val_acc: 0.0157
Epoch 3/30
35000/35000 [==============================] - 31s 891us/step - loss: 0.0099 - acc: 0.0149 - val_loss: 0.0099 - val_acc: 0.0164
Epoch 4/30
35000/35000 [==============================] - 31s 891us/step - loss: 0.0099 - acc: 0.0150 - val_loss: 0.0099 - val_acc: 0.0176
Epoch 5/30
35000/35000 [==============================] - 31s 889us/step - loss: 0.0099 - acc: 0.0166 - val_loss: 0.0099 - val_acc: 0.0192
Epoch 6/30
35000/35000 [==============================] - 31s 889us/step - loss: 0.0099 - acc: 0.0212 - val_loss: 0.0099 - val_acc: 0.0246
Epoch 7/30
35000/35000 [==============================] - 31s 892us/step

When trained with 100 epochs, the accuracy of the G1 was better than the G2, which was really weird...

In [29]:
model_G2.save("model_G2_35k_sgd.h5")

In [9]:
model_G2 = keras.models.load_model('model_G2_15k.h5')

W0728 19:49:50.543153 139699900229376 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 19:49:50.596954 139699900229376 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 19:49:50.629657 139699900229376 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 19:49:50.630287 139699900229376 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The 

fix layers

In [30]:
for layer in model_G2.layers[:20]:
    layer.trainable=False
for layer in model_G2.layers[20:]:
    layer.trainable=True

Retrain with 15K

In [15]:
compile_model(model_G2)

In [31]:
model_G2.fit(x_train_local, y_train_local,
              batch_size=batch_size,
              epochs=30,
              validation_data=(x_test, y_test_cat),
              shuffle=True)

Train on 15000 samples, validate on 10000 samples
Epoch 1/30
  160/15000 [..............................] - ETA: 16s - loss: 0.0097 - acc: 0.0688

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


15000/15000 [==============================] - 15s 970us/step - loss: 0.0097 - acc: 0.0618 - val_loss: 0.0096 - val_acc: 0.0638
Epoch 2/30
15000/15000 [==============================] - 14s 963us/step - loss: 0.0097 - acc: 0.0633 - val_loss: 0.0096 - val_acc: 0.0629
Epoch 3/30
15000/15000 [==============================] - 14s 959us/step - loss: 0.0096 - acc: 0.0665 - val_loss: 0.0097 - val_acc: 0.0680
Epoch 4/30
15000/15000 [==============================] - 14s 965us/step - loss: 0.0096 - acc: 0.0667 - val_loss: 0.0097 - val_acc: 0.0638
Epoch 5/30
15000/15000 [==============================] - 14s 958us/step - loss: 0.0096 - acc: 0.0691 - val_loss: 0.0096 - val_acc: 0.0676
Epoch 6/30
15000/15000 [==============================] - 14s 959us/step - loss: 0.0096 - acc: 0.0726 - val_loss: 0.0097 - val_acc: 0.0664
Epoch 7/30
15000/15000 [==============================] - 14s 959us/step - loss: 0.0096 - acc: 0.0752 - val_loss: 0.0097 - val_acc: 0.0681
Epoch 8/30
15000/15000 [==============

In [32]:
model_G2.save("model_G2_35k_15k_sgd.h5")

Sanity Test: If layer not fixed?

In [22]:
model_G2_layer_not_fixed = keras.models.load_model("model_G2_epoch_30.h5")

In [23]:
model_G2_layer_not_fixed.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [1]:
model_G2_layer_not_fixed.fit(x_train_local, y_train_local,
              batch_size=batch_size,
              epochs=30,
              validation_data=(x_test, y_test_cat),
              shuffle=True)

NameError: name 'model_G2_layer_not_fixed' is not defined

Only a small change...

Why a sudden decrease in accuaracy when training started?
It wasn't fair for G2 to go through 60 epochs total(first training 30 + the second 30)? 
G1 and G2 have to go through the same number of mini batches

#### Global (3): Train globally on 35K + fix layers + distribute 15K randomly + train & aggregate (Federated Learning)

In [30]:
# FL Setting
num_users=2

In [31]:
# list of split data
x_data_split = np.split(x_train_local, num_users)
y_data_split = np.split(y_train_local, num_users)

In [32]:
# Load pre-trained model
pretrained_model = keras.models.load_model('model_G2_35k_sgd.h5')

In [33]:
# Populate users with models
users_models = [keras.models.clone_model(pretrained_model) for i in range(num_users)]

In [34]:
for m in users_models:
    m.set_weights(pretrained_model.get_weights())

In [35]:
for m in users_models:
    compile_model(m)

In [36]:
users_models[0].evaluate(x_test, y_test_cat)

10000/10000 [==============================] - 29s 3ms/step


[0.009655948668718339, 0.0676]

In [37]:
local_update_history = []
for i, m in enumerate(users_models):
    print("fitting %dth model" %(i+1))
    local_update_history.append(
        m.fit(x_data_split[i], y_data_split[i],
              batch_size=32,
              epochs=30,
              validation_data=(x_test, y_test_cat),
              shuffle=True)
    )

fitting 1th model
Train on 7500 samples, validate on 10000 samples
Epoch 1/30
7500/7500 [==============================] - 41s 5ms/step - loss: 0.0097 - acc: 0.0597 - val_loss: 0.0096 - val_acc: 0.0653
Epoch 2/30
7500/7500 [==============================] - 17s 2ms/step - loss: 0.0097 - acc: 0.0621 - val_loss: 0.0096 - val_acc: 0.0651
Epoch 3/30
7500/7500 [==============================] - 17s 2ms/step - loss: 0.0096 - acc: 0.0668 - val_loss: 0.0096 - val_acc: 0.0676
Epoch 4/30
7500/7500 [==============================] - 16s 2ms/step - loss: 0.0096 - acc: 0.0693 - val_loss: 0.0097 - val_acc: 0.0648
Epoch 5/30
7500/7500 [==============================] - 14s 2ms/step - loss: 0.0096 - acc: 0.0696 - val_loss: 0.0097 - val_acc: 0.0558
Epoch 6/30
7500/7500 [==============================] - 16s 2ms/step - loss: 0.0096 - acc: 0.0717 - val_loss: 0.0097 - val_acc: 0.0629
Epoch 7/30
7500/7500 [==============================] - 16s 2ms/step - loss: 0.0095 - acc: 0.0737 - val_loss: 0.0097 - val_

In [38]:
weights = [m.get_weights() for m in users_models]

In [39]:
# FederatedAggregation
new_weights = list()

for weights_list_tuple in zip(*weights):
    new_weights.append(
        [np.array(weights_).mean(axis=0)\
            for weights_ in zip(*weights_list_tuple)])

In [40]:
aggr_model = keras.models.clone_model(pretrained_model)
aggr_model.set_weights(new_weights)

partial aggregation

In [41]:
compile_model(aggr_model)
aggr_model.evaluate(x_test, y_test_cat)

10000/10000 [==============================] - 24s 2ms/step


[0.00967746299803257, 0.0609]

In [42]:
aggr_model.save("model_G3_users_2(1).h5")

In [43]:
model_local_test = keras.models.load_model("model_G3_users_2(1).h5")

In [44]:
compile_model(model_local_test)
model_local_test.evaluate(x_test, y_test_cat)

10000/10000 [==============================] - 26s 3ms/step


[0.00967746299803257, 0.0609]

#### Global (4): Train globally on 35K + fix layers + distribute 15K NON-randomly + train & aggregate (Federated Learning)

In [83]:
# FL Setting
num_users=10

##### Users get datasets corresponding to (num_class/num_users) classes each

In [84]:
x_train_local[0].shape

(32, 32, 3)

In [106]:
y_train_local.shape

(15000, 100)

In [107]:
x_data_split = [np.empty([0,32,32,3]) for _ in range(num_users)]
y_data_split = [np.empty([0,100]) for _ in range(num_users)]

In [110]:
for i in range(y_train_local.shape[0]):
    if (i%1000 == 0):
        print("%d complete" %i)
    user_num = (int) (y_train_local_notcat[i][0] / (100 / num_users))
    x_data_split[user_num] = \
        np.concatenate((x_data_split[user_num], np.expand_dims(x_train_local[i], axis=0)), axis=0)
    y_data_split[user_num] = \
        np.concatenate((y_data_split[user_num], np.expand_dims(y_train_local[i], axis=0)), axis=0)

0 complete
1000 complete
2000 complete
3000 complete
4000 complete
5000 complete
6000 complete
7000 complete
8000 complete
9000 complete
10000 complete
11000 complete
12000 complete
13000 complete
14000 complete


In [111]:
# Load pre-trained model
pretrained_model = keras.models.load_model('model_G2_35k_sgd.h5')

In [112]:
# Populate users with models
users_models = [keras.models.clone_model(pretrained_model) for i in range(num_users)]

In [113]:
for m in users_models:
    m.set_weights(pretrained_model.get_weights())

In [114]:
for m in users_models:
    compile_model(m)

In [115]:
users_models[0].evaluate(x_test, y_test_cat)

10000/10000 [==============================] - 20s 2ms/step


[0.009655948668718339, 0.0676]

In [116]:
local_update_history = []

print_datetime()

for i, m in enumerate(users_models):
    print("model #%d" %(i+1))
    local_update_history.append(
        m.fit(x_data_split[i], y_data_split[i],
              batch_size=32,
              epochs=30,
              validation_data=(x_test, y_test_cat),
              shuffle=True)
    )

Training date and time : 
2019-08-02 17:49:49
model #1
augmenting dataset
Train on 4527 samples, validate on 10000 samples
Epoch 1/30
4527/4527 [==============================] - 25s 6ms/step - loss: 0.0096 - acc: 0.0769 - val_loss: 0.0097 - val_acc: 0.0580
Epoch 2/30
4527/4527 [==============================] - 7s 2ms/step - loss: 0.0095 - acc: 0.0987 - val_loss: 0.0097 - val_acc: 0.0556
Epoch 3/30
4527/4527 [==============================] - 7s 2ms/step - loss: 0.0094 - acc: 0.1124 - val_loss: 0.0097 - val_acc: 0.0559
Epoch 4/30
4527/4527 [==============================] - 7s 2ms/step - loss: 0.0094 - acc: 0.1257 - val_loss: 0.0097 - val_acc: 0.0557
Epoch 5/30
4527/4527 [==============================] - 10s 2ms/step - loss: 0.0093 - acc: 0.1387 - val_loss: 0.0097 - val_acc: 0.0561
Epoch 6/30
4527/4527 [==============================] - 10s 2ms/step - loss: 0.0093 - acc: 0.1498 - val_loss: 0.0098 - val_acc: 0.0561
Epoch 7/30
4527/4527 [==============================] - 9s 2ms/step - 

Epoch 30/30
1454/1454 [==============================] - 7s 5ms/step - loss: 0.0059 - acc: 0.5420 - val_loss: 0.0127 - val_acc: 0.0344
model #5
augmenting dataset
Train on 1503 samples, validate on 10000 samples
Epoch 1/30
1503/1503 [==============================] - 27s 18ms/step - loss: 0.0096 - acc: 0.0705 - val_loss: 0.0097 - val_acc: 0.0609
Epoch 2/30
1503/1503 [==============================] - 5s 3ms/step - loss: 0.0094 - acc: 0.1457 - val_loss: 0.0098 - val_acc: 0.0365
Epoch 3/30
1503/1503 [==============================] - 6s 4ms/step - loss: 0.0093 - acc: 0.1956 - val_loss: 0.0101 - val_acc: 0.0261
Epoch 4/30
1503/1503 [==============================] - 6s 4ms/step - loss: 0.0092 - acc: 0.2122 - val_loss: 0.0101 - val_acc: 0.0216
Epoch 5/30
1503/1503 [==============================] - 7s 4ms/step - loss: 0.0091 - acc: 0.2236 - val_loss: 0.0101 - val_acc: 0.0193
Epoch 6/30
1503/1503 [==============================] - 6s 4ms/step - loss: 0.0090 - acc: 0.2249 - val_loss: 0.0100 

Epoch 29/30
1475/1475 [==============================] - 5s 3ms/step - loss: 0.0069 - acc: 0.4176 - val_loss: 0.0118 - val_acc: 0.0344
Epoch 30/30
1475/1475 [==============================] - 5s 3ms/step - loss: 0.0068 - acc: 0.4380 - val_loss: 0.0122 - val_acc: 0.0349
model #9
augmenting dataset
Train on 1496 samples, validate on 10000 samples
Epoch 1/30
1496/1496 [==============================] - 28s 19ms/step - loss: 0.0093 - acc: 0.1384 - val_loss: 0.0100 - val_acc: 0.0432
Epoch 2/30
1496/1496 [==============================] - 7s 5ms/step - loss: 0.0088 - acc: 0.2039 - val_loss: 0.0102 - val_acc: 0.0334
Epoch 3/30
1496/1496 [==============================] - 6s 4ms/step - loss: 0.0085 - acc: 0.2400 - val_loss: 0.0101 - val_acc: 0.0325
Epoch 4/30
1496/1496 [==============================] - 6s 4ms/step - loss: 0.0083 - acc: 0.2894 - val_loss: 0.0101 - val_acc: 0.0319
Epoch 5/30
1496/1496 [==============================] - 5s 3ms/step - loss: 0.0082 - acc: 0.3041 - val_loss: 0.0103

In [117]:
weights = [m.get_weights() for m in users_models]

In [118]:
# FederatedAggregation
new_weights = list()

for weights_list_tuple in zip(*weights):
    new_weights.append(
        [np.array(weights_).mean(axis=0)\
            for weights_ in zip(*weights_list_tuple)])

In [119]:
aggr_model = keras.models.clone_model(pretrained_model)
aggr_model.set_weights(new_weights)

partial aggregation

In [120]:
compile_model(aggr_model)
aggr_model.evaluate(x_test, y_test_cat)

10000/10000 [==============================] - 23s 2ms/step


[0.009900018228590489, 0.01]

In [121]:
aggr_model.save("model_G4_users_10.h5")

In [122]:
model_local_test = keras.models.load_model("model_G4_users_10.h5")

In [123]:
compile_model(model_local_test)
model_local_test.evaluate(x_test, y_test_cat)

10000/10000 [==============================] - 23s 2ms/step


[0.009900018228590489, 0.01]

In [125]:
pickle.dump( local_update_history, open( "update_history_G4_users_10.p", "wb" ) )

KeyboardInterrupt: 


### Global (4): Train globally on 35K + fix layers + distribute 15K NON-randomly + train & aggregate (Federated Learning)
#### 2 users

In [126]:
# FL Setting
num_users = 2

##### Users get datasets corresponding to (num_class/num_users) classes each

In [127]:
x_train_local[0].shape

(32, 32, 3)

In [128]:
y_train_local.shape

(15000, 100)

In [129]:
x_data_split = [np.empty([0,32,32,3]) for _ in range(num_users)]
y_data_split = [np.empty([0,100]) for _ in range(num_users)]

In [ ]:
for i in range(y_train_local.shape[0]):
    if (i%1000 == 0):
        print("%d complete" %i)
    user_num = (int) (y_train_local_notcat[i][0] / (100 / num_users))
    x_data_split[user_num] = \
        np.concatenate((x_data_split[user_num], np.expand_dims(x_train_local[i], axis=0)), axis=0)
    y_data_split[user_num] = \
        np.concatenate((y_data_split[user_num], np.expand_dims(y_train_local[i], axis=0)), axis=0)

0 complete
1000 complete
2000 complete
3000 complete
4000 complete
5000 complete
6000 complete
7000 complete
8000 complete
9000 complete
10000 complete
11000 complete
12000 complete
13000 complete


In [ ]:
# Load pre-trained model
pretrained_model = keras.models.load_model('model_G2_35k_sgd.h5')

In [ ]:
# Populate users with models
users_models = [keras.models.clone_model(pretrained_model) for i in range(num_users)]

In [ ]:
for m in users_models:
    m.set_weights(pretrained_model.get_weights())

In [ ]:
for m in users_models:
    compile_model(m)

In [ ]:
users_models[0].evaluate(x_test, y_test_cat)

In [ ]:
local_update_history = []

print_datetime()

for i, m in enumerate(users_models):
    print("model #%d" %(i+1))
    local_update_history.append(
        m.fit(x_data_split[i], y_data_split[i],
              batch_size=32,
              epochs=30,
              validation_data=(x_test, y_test_cat),
              shuffle=True)
    )

In [ ]:
weights = [m.get_weights() for m in users_models]

In [ ]:
# FederatedAggregation
new_weights = list()

for weights_list_tuple in zip(*weights):
    new_weights.append(
        [np.array(weights_).mean(axis=0)\
            for weights_ in zip(*weights_list_tuple)])

In [ ]:
aggr_model = keras.models.clone_model(pretrained_model)
aggr_model.set_weights(new_weights)

partial aggregation

In [ ]:
compile_model(aggr_model)
aggr_model.evaluate(x_test, y_test_cat)

In [ ]:
aggr_model.save("model_G4_users_2.h5")

In [ ]:
model_local_test = keras.models.load_model("model_G4_users_2.h5")

In [ ]:
compile_model(model_local_test)
model_local_test.evaluate(x_test, y_test_cat)

In [ ]:
pickle.dump( local_update_history, open( "update_history_G4_users_2.p", "wb" ) )